# 03 Safe transfer with staged adaptation
Hypothesis: staged adaptation preserves source knowledge while improving target accuracy.

### Expected Outcome
Gradual unfreezing should keep higher source retention than naive fine-tuning.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
methods = ['feature_extraction', 'gradual_unfreeze', 'naive_finetune']
LAB.run('transfer_safe_vs_naive.yaml', '03_safe', fast_dev_run=FAST_DEV_RUN)
frames = LAB.load_methods(methods)
safe = frames['gradual_unfreeze']
naive = frames['naive_finetune']

In [ ]:
fig, ax = plt.subplots(figsize=(6.6, 3.7))
for m, df in frames.items():
    ax.plot(df['epoch'], df['target_test_acc'], marker='o', label=m)
ax.set_title('Target accuracy: safe vs aggressive')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '03_safe_vs_naive_target_acc.png')

fig, ax = plt.subplots(figsize=(6.6, 3.7))
for m, df in frames.items():
    ax.plot(df['epoch'], df['source_retention_acc'], marker='o', label=m)
ax.set_title('Source retention: safe vs aggressive')
ax.set_xlabel('epoch')
ax.set_ylabel('source_retention_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '03_safe_vs_naive_retention.png')

In [ ]:
comparison = safe[['epoch']].copy()
comparison['delta_target_acc_safe_minus_naive'] = safe['target_test_acc'] - naive['target_test_acc']
comparison['delta_retention_safe_minus_naive'] = safe['source_retention_acc'] - naive['source_retention_acc']
comparison.tail()

### Practical Signal
Positive retention delta with equal-or-better target accuracy indicates safer transfer.